In [5]:
import openai
openai.api_key_path = '/home/jong/.openai_key'

In [76]:
class Chat:
    def __init__(self, topic, podcast="award winning NPR", max_length=4096//4, hosts=['Tom', 'Jen']):
        system = f"You are an {podcast} podcast with hosts {hosts[0]} and {hosts[1]}."
        self._system = system
        self._topic = topic
        self._max_length = max_length
        self._hosts = hosts
        self._history = [
            {"role": "system", "content": self._system},
            {"role": "user", "content": f"Generate a podcast episode about {topic}, including history and other fun facts. Reference published scientific journals."},
        ]

    @classmethod
    def num_tokens_from_messages(cls, messages, model="gpt-3.5-turbo"):
        """Returns the number of tokens used by a list of messages."""
        encoding = tiktoken.encoding_for_model(model)
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
        
    def message(self, next_msg=None):
        # TODO: Optimize this if slow through easy caching
        while len(self._history) > 1 and self.num_tokens_from_messages(self._history) > self._max_length:
            self._history.pop(1)
        if next_msg is not None:
            self._history.append({"role": "user", "content": next_msg})
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self._history,
        )
        text = resp.choices[0].message.content
        self._history.append({"role": "assistant", "content": text})
        return text

    def text2speech(self, text, spacing_ms=350):
        with tempfile.TemporaryDirectory() as tmpdir:
            i = 0
            def write_audio(msg, **kwargs):
                from gtts import gTTS
                import tempfile
                s = gTTS(msg, **kwargs)
                nonlocal i
                s.save(f'{tmpdir}/{i}.mp3')
                i += 1
                return f'{i-1}.mp3'

            text = text.replace('\n', '!!!LINEBREAK!!!').replace('\\', '')
            # Build text one at a time
            currline, currname = "", self._hosts[0]
            name2tld = {self._hosts[0]: 'co.uk', self._hosts[1]: 'co.za'}
            for line in text.split("!!!LINEBREAK!!!"):
                if not line.strip(): continue
                if line.startswith(f"{self._hosts[0]}: ") or line.startswith(f"{self._hosts[1]}: "):
                    if currline:
                        write_audio(currline, lang='en', tld=name2tld[currname])
                    currline = line[4:]
                    currname = line[:3]
                else:
                    currline += line
            if currline:
                write_audio(currline, lang='en', tld=name2tld[currname])
            # Concat files
            from pydub import AudioSegment
            total_sound = None
            for mp3_i in range(i):
                sound = AudioSegment.from_mp3(f"{tmpdir}/{mp3_i}.mp3")
                if total_sound is None:
                    total_sound = sound
                else:
                    total_sound = total_sound + AudioSegment.silent(duration=spacing_ms) + sound
                
            # writing mp3 files is a one liner
            total_sound.export(f"{tmpdir}/total.mp3", format="mp3")
            display(IPython.display.Audio(f"{tmpdir}/total.mp3", autoplay=True))
            
    def step(self, msg=None):
        msg = self.message(msg)
        self.text2speech(msg)
        return msg

In [48]:
chat = Chat('Yogurt')
msg = chat.message()
Chat.text2speech(msg)

In [73]:
chat = Chat("Sloths", hosts=['Noa', 'Guy'])
chat.step()

'[Opening music]\n\nNoa: Welcome to the show, I’m Noa.\n\nGuy: And I’m Guy. Today we’re talking about a creature that moves at a glacial pace but has captured the hearts and imaginations of people all over the world. We’re talking, of course, about…\n\nNoa and Guy (in unison): Sloths!\n\nGuy: Sloths are fascinating creatures, with their seemingly lackadaisical lifestyle and cute, often goofy faces. But there’s so much more to these animals than their outward appearance.\n\nNoa: That’s right, Guy. According to the International Union for Conservation of Nature, there are six different species of sloths, four of which are found in Central and South America.\n\nGuy: Let’s start with some history. Did you know that sloths were first documented by European explorers in the 15th and 16th centuries?\n\nNoa: Yeah, and fun fact: they were initially mistaken for monkeys. It wasn’t until the 18th century that scientists began to recognize them as a distinct group of animals.\n\nGuy: The two-toed 

In [74]:
chat.step('Make another funny episode about Sloths. Do not repeat any information from before.')

'[Opening music]\n\nNoa: Welcome to the show, I’m Noa.\n\nGuy: And I’m Guy. Today, we’re talking about the laziest, sleepiest, and cutest animals on the planet: sloths!\n\nNoa: Speaking of laziness, did you know that sloths sleep for up to 15 hours a day? That’s nearly two-thirds of their entire life!\n\nGuy: Don’t forget about their slow movement. Sloths move so slowly that scientists once clocked one traveling just seven feet in a minute.\n\nNoa: And did you know that sloths have a symbiotic relationship with moths? Sloths grow algae on their fur, which the moths then eat.\n\nGuy: Wait, what? Algae? On their fur? That’s disgusting!\n\nNoa: Hey, don’t be sloth-shaming. Besides, if you think that’s gross, just wait until you hear about the sloth’s digestive system.\n\nGuy: I don’t think I want to hear this.\n\nNoa: Sloths only poop once a week, and it can take them up to a month to digest one meal.\n\nGuy: Ew, that’s seriously gross.\n\nNoa: Hey, you’re the one who wanted to talk about

In [50]:
msg = chat.message('Tell me other things about yogurt and make it funny')
Chat.text2speech(msg)

In [51]:
chat._history

[{'role': 'system',
  'content': 'You are an award winning NPR podcast with hosts Tom and Jen.'},
 {'role': 'user',
  'content': 'Generate a podcast episode about Yogurt, including history and other fun facts. Reference published scientific journals.'},
 {'role': 'assistant',
  'content': 'Tom: Welcome back to our podcast, today we are discussing Yogurt! A dairy product that has been around for thousands of years.\n\nJen: That’s right, Tom. In fact, the origins of yogurt can be traced back to the Neolithic period over 10,000 years ago. Nomadic tribes in Central Asia discovered that milk left to ferment in animal skins preserved the milk and created a tasty and nutritious sour milk product, which we now know as yogurt.\n\nTom: That’s fascinating. I had no idea yogurt had such a rich history. What are some of the health benefits of eating yogurt?\n\nJen: Well, according to research published in the American Journal of Clinical Nutrition, yogurt is a great source of calcium, protein, and 

In [10]:
"""
TODO:
    Make web server and send mp3 to frontend
    Make frontend and play results
"""

'\nTODO:\n'